In [464]:
import pandas as pd
from utils import extract_val_from_volume

In [465]:
csv_file_1 = '../data/2019-bypass-overflow.csv'
csv_file_2 = '../data/2020-bypass-overflow.csv'
csv_file_3 = '../data/2021-bypass-overflow.csv'
csv_file_4 = '../data/2022_overflow_data.csv'
csv_locations = '../data/locations.csv'
table_name = 'overflows_staging'

DEFAULT_COLUMNS = {
	'location',
	'start_date',
	'start_time',
	'stop_date',
	'stop_time',
	'duration',
	'volume',
	'disinfection',
	'reason_code'
}

df1 = pd.read_csv(csv_file_1)
df2 = pd.read_csv(csv_file_2)
df3 = pd.read_csv(csv_file_3)
df4 = pd.read_csv(csv_file_4)
locations = pd.read_csv(csv_locations)

In [466]:
df_three = pd.concat([df1, df2, df3])
df_three = df_three.rename(columns={'end_date': 'stop_date', 'end_time': 'stop_time'})
df_three = df_three[DEFAULT_COLUMNS]

In [467]:
df4['start_date'] = pd.to_datetime(df4['initiation']).dt.date
df4['start_time'] = pd.to_datetime(df4['initiation']).dt.time
df4['stop_date'] = pd.to_datetime(df4['completion']).dt.date
df4['stop_time'] = pd.to_datetime(df4['completion']).dt.time

df4.rename(columns={'cso_outfall': 'location'}, inplace=True)
df4['disinfection'] = ''
df4['reason_code'] = ''
df4['volume'] = df4['volume'].apply(extract_val_from_volume)

df4 = pd.merge(df4, locations, on=['location'])
df4 = df4.drop(['location'], axis=1)

df4.rename(columns={'id': 'location'}, inplace=True)
df4.replace({'duration': ' hrs'}, {'duration': ''}, regex=True, inplace=True)

df4 = df4[DEFAULT_COLUMNS]

In [468]:
# COMBINE ALL HISTORICAL TABLES

result = pd.concat([df_three, df4])

In [469]:
result.to_csv('../data/historical_overflows.csv', index=False)